# Тональность отзывов

Сайт -- bookmix, материал -- рецензии, потому что они длинные и интересно, что получится

### 0. Улучшения

Хорошо бы подумать о них до написания основного кода.
1. если взять больше материала, точность должна быть выше. 
2. однозвёздочных рецензий мало. можно добавить к ним двухзвёздочные и посмотреть, что получится. 

### 1. Скачивание отзывов

Вначале соберём отзывы. Поскольку можно все 70 нужных ссылок получить одним обходом сайта, писать для этого функцию нет необходимости. 

In [2]:
from tqdm import tqdm
from bs4 import BeautifulSoup
import requests
from fake_useragent import UserAgent

ua = UserAgent(verify_ssl=False)
session = requests.session()

In [3]:
negative_words = []
positive_words = []
begin = 0
for i in tqdm(range(500)):#while len(negative_words) < 35 or len(positive_words) < 35:
    begin += 10
    url = f'https://bookmix.ru/reviews.phtml?option=all&begin={begin}&num_point=10&num_points=10'
    req = session.get(url, headers={'User-Agent': ua.random})
    req.encoding = 'utf-8'
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    titles = soup.find_all('div', {'class': 'universal-blocks'})
    for title in titles:
        rate = title.find('div', {'class': 'rating'})
        if rate:
            rate = rate.attrs['class'][-1][-1]
            if rate == '1' or rate == '2':
                href = title.find('h5').find('a').attrs['href']
                negative_words.append(href)
            elif rate == '5':# and len(positive_words) < 35:
                href = title.find('h5').find('a').attrs['href']
                positive_words.append(href)

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [10:29<00:00,  1.26s/it]


In [4]:
len(positive_words)

2645

In [5]:
len(negative_words)

291

Поскольку количество негативных и позитивных отзывов очень разное, можно взять по 260 тех и других на обучение и по 30 на проверку

### 2. Токенизация и лемматизация

In [6]:
from nltk import word_tokenize
from collections import Counter
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [20]:
def get_texts(hrefs):
    words = []
    for href in hrefs: #tqdm(hrefs):
        href = f'https://bookmix.ru/{href}'
        req = session.get(href, headers={'User-Agent': ua.random})
        req.encoding = 'utf-8'
        req = req.text
        req = BeautifulSoup(req, 'html.parser')
        text = req.find('div', {'class': 'universal-blocks-content'}).text.lower()
        text = word_tokenize(text)
        for word in text:
            if word.isalpha():
                words.append(morph.parse(word)[0][2]) #добаляем в словарь лемму
    words = dict(Counter(words))
    return words

In [8]:
positive_dict = get_texts(positive_words[:260])
negative_dict = get_texts(negative_words[:260])

100%|████████████████████████████████████████████████████████████████████████████████| 260/260 [06:49<00:00,  1.58s/it]


In [9]:
len(positive_dict)

13816

In [10]:
len(negative_dict)

14778

### 3. Множества и исключение шума

In [32]:
def make_set(some_dict):
    word_set = []
    for word, freq in some_dict.items():
        if freq > 2:
            word_set.append(word)
    word_set = set(word_set)
    return word_set

In [33]:
positive_set = make_set(positive_dict) 
negative_set = make_set(negative_dict)

In [34]:
all_set = positive_set & negative_set
positive_set = positive_set.difference(all_set)
negative_set = negative_set.difference(all_set)

### 4. Функция для определения тональности отзыва

In [35]:
def tone(href):
    pos = 0
    neg = 0
    words = get_texts([href,])
    for word, freq in words.items():
        if word in positive_set:
            pos += 1*freq
        elif word in negative_set:
            neg += 1*freq
    if pos > neg:
        result = 'pos'
    elif pos < neg:
        result = 'neg'
    else:
        result = 'nonloso'
    return result

### 5. Accuracy

In [36]:
from sklearn.metrics import accuracy_score

In [37]:
def detect_tone(positive_hrefs, negative_hrefs):
    detected_tone = []
    real_tone = []
    for text in positive_hrefs:
        detected_tone.append(tone(text))
        real_tone.append('pos')
    for text in negative_hrefs:
        detected_tone.append(tone(text))
        real_tone.append('neg')
    return accuracy_score(detected_tone, real_tone)

In [38]:
detect_tone(positive_words[260:292], negative_words[260:])

0.7619047619047619

Так себе точность, но:
1) с откидыванием слов, которые встретились один или три раза, всё становится хуже. 
2) Можно было бы увеличить объём текстов за счёт обхода всего корпуса, но в задании и так сказано, что минимум -- по 30.
3) Книжные рецензии -- тексты длинные, и предположительно могут быть устроены сложнее, чем короткие отзывы
4) Можно наткнуться на сарказм не слишком редко, а такие штуки плохо поддаются анализу на основе только слов